In [1]:
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers, activations, regularizers
from tensorflow.keras.datasets import mnist 
import tensorflow_hub as hub

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable = True)

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32')/255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32')/255.0

In [7]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size = 3):
        super(CNNBlock, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size, padding = 'same')
        self.bn = layers.BatchNormalization()
        # self.relu = keras.activations.relu
        self.relu = layers.ReLU()
        
    def call(self, input_tensor, training = False):
        x = self.conv1(input_tensor)
        x = self.bn(x, training = training)
        x = self.relu(x)
        return x

In [8]:
inp = tf.keras.Input(shape = (28,28,1))
conv1 = CNNBlock(32)(inp)
conv2 = CNNBlock(64)(conv1)
conv3 = CNNBlock(128)(conv2)
gap = layers.GlobalAveragePooling2D()(conv3) # Produces (batch_size, 128)
# flatten = layers.Flatten()(conv3) (Would produce (batch_size, H*W*128))
dense1 = layers.Dense(128, activation = 'relu')(gap)
dense2 = layers.Dense(10)(dense1)

model = Model(inputs = [inp], outputs = [dense2])
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 cnn_block_1 (CNNBlock)      (None, 28, 28, 32)        448       
                                                                 
 cnn_block_2 (CNNBlock)      (None, 28, 28, 64)        18752     
                                                                 
 cnn_block_3 (CNNBlock)      (None, 28, 28, 128)       74368     
                                                                 
 global_average_pooling2d (G  (None, 128)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                             

In [9]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [10]:
model.fit(x_train, y_train, batch_size = 64, verbose = 2, epochs = 3)
model.evaluate(x_test, y_test, batch_size = 64, verbose = 2)

Epoch 1/3
938/938 - 23s - loss: 0.4543 - accuracy: 0.8724 - 23s/epoch - 24ms/step
Epoch 2/3
938/938 - 16s - loss: 0.1421 - accuracy: 0.9583 - 16s/epoch - 17ms/step
Epoch 3/3
938/938 - 16s - loss: 0.1129 - accuracy: 0.9662 - 16s/epoch - 17ms/step
157/157 - 1s - loss: 6.1408 - accuracy: 0.2180 - 1s/epoch - 8ms/step


[6.140760898590088, 0.21799999475479126]

In [14]:
model.save('pretrained/BasicCNN/')
model_new = keras.models.load_model("pretrained/BasicCNN/", custom_objects = {'CNNBlock': CNNBlock})
print(model_new.summary())

INFO:tensorflow:Assets written to: pretrained/BasicCNN/assets


INFO:tensorflow:Assets written to: pretrained/BasicCNN/assets


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 cnn_block_1 (CNNBlock)      (None, 28, 28, 32)        448       
                                                                 
 cnn_block_2 (CNNBlock)      (None, 28, 28, 64)        18752     
                                                                 
 cnn_block_3 (CNNBlock)      (None, 28, 28, 128)       74368     
                                                                 
 global_average_pooling2d (G  (None, 128)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                             

In [15]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.conv1 = CNNBlock(channels[0])
        self.conv2 = CNNBlock(channels[1])
        self.conv3 = CNNBlock(channels[2])
        self.pool1 = layers.MaxPooling2D()
        self.identity = layers.Conv2D(channels[1], 1, padding = 'same')

    def call(self, input_tensor, training = False):
        op1 = self.conv1(input_tensor, training = training)
        op2 = self.conv2(op1, training = training)
        op3 = self.conv3(op2 + self.identity(input_tensor), training = training) 
        pool = self.pool1(op3)
        
        return pool

In [16]:
channels1 = [32, 32, 128]
channels2 = [128, 128, 256]
channels3 = [128, 256, 512]

inp = keras.Input(shape = (28, 28, 1))
res1 = ResBlock(channels1)(inp)
res2 = ResBlock(channels2)(res1)
res3 = ResBlock(channels3)(res2)
gap = layers.GlobalAveragePooling2D()(res3)
# flatten = layers.Flatten(gap)
dense = layers.Dense(10)(gap)

model = Model(inputs = [inp, ], outputs = [dense, ])
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 res_block (ResBlock)        (None, 14, 14, 128)       47392     
                                                                 
 res_block_1 (ResBlock)      (None, 7, 7, 256)         608896    
                                                                 
 res_block_2 (ResBlock)      (None, 3, 3, 512)         1839744   
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                           

In [17]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [18]:
model.fit(x_train, y_train, epochs = 1, verbose = 2, batch_size = 64)
model.evaluate(x_test, y_test, verbose = 2, batch_size = 64)

938/938 - 44s - loss: 0.0864 - accuracy: 0.9745 - 44s/epoch - 47ms/step
157/157 - 3s - loss: 0.0605 - accuracy: 0.9808 - 3s/epoch - 19ms/step


[0.06048215180635452, 0.9807999730110168]

In [19]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype('float32')/255.0
x_test = x_test.reshape(-1, 28 * 28).astype('float32')/255.0

In [20]:
class CustomLayer(layers.Layer):
    def __init__(self, output_dim, input_dim):
        super(CustomLayer, self).__init__()
        self.w = self.add_weight(
            name = 'weight-x',
            shape = (input_dim, output_dim),
            initializer = 'random_normal',
            trainable = True,
        )
        self.b = self.add_weight(
            name = 'bias-x',
            shape = (output_dim, ),
            initializer = 'zeros',
            trainable = True,
        )
    def call(self, input_tensor):
        op = tf.matmul(input_tensor, self.w) + self.b
        return op

In [21]:
class CustomModel(keras.Model):
    def __init__(self, num_classes = 10):
        super(CustomModel, self).__init__()
        self.dense1 = CustomLayer(64, 784)
        self.dense2 = CustomLayer(num_classes, 64)

    def call(self, input_tensor):
        layer1 = tf.nn.relu(self.dense1(input_tensor))
        return self.dense2(layer1)

In [22]:
model_custom = CustomModel()
model_custom.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy'],
)

model_custom.fit(x_train, y_train, verbose = 2, batch_size = 32, epochs = 1)
model_custom.evaluate(x_test, y_test, verbose = 2, batch_size = 32)

1875/1875 - 9s - loss: 0.3447 - accuracy: 0.9055 - 9s/epoch - 5ms/step
313/313 - 2s - loss: 0.1843 - accuracy: 0.9431 - 2s/epoch - 5ms/step


[0.18426132202148438, 0.9430999755859375]

Saving and Loading Model (Serialization of Model - saved as a data structure) and Model weights

Saving the model also saves the:
1. Model Weigths
2. Model Architecture
3. Training Configuration (The configs the model was compiled with)
4. Optimizer and states

In [23]:
# Sequential model saves work only with Sequential and Functional model saves work only with Functional model.

# model.save__weights('saved_model/', save_format = 'h5')
# model = model.load_weights('saved_model/')

In [24]:
#Saving the entire model with all configs: 
# (Works with Sequential, Functional and Subclassing)

# model.save('entire_saved_model/')
# model = keras.models.load_model('entire_saved_model/')

Using Pretrained Models and Freeing layers

In [31]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32')/255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32')/255.0

In [32]:
# Using a manually made pretrained model:
model_old = keras.models.load_model('pretrained/BasicCNN/', custom_objects = {'CNNBlock': CNNBlock})
print(model_old.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 cnn_block_1 (CNNBlock)      (None, 28, 28, 32)        448       
                                                                 
 cnn_block_2 (CNNBlock)      (None, 28, 28, 64)        18752     
                                                                 
 cnn_block_3 (CNNBlock)      (None, 28, 28, 128)       74368     
                                                                 
 global_average_pooling2d (G  (None, 128)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                             

In [33]:
base_input = model_old.layers[0].input
base_output = model_old.layers[-2].output
classification = layers.Dense(10)(base_output)
model_modified = Model(inputs = [base_input, ], outputs = [classification, ])

for layer in model_modified.layers[:-1]:
    layer.trainable == False

print(model_modified.summary())

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 cnn_block_1 (CNNBlock)      (None, 28, 28, 32)        448       
                                                                 
 cnn_block_2 (CNNBlock)      (None, 28, 28, 64)        18752     
                                                                 
 cnn_block_3 (CNNBlock)      (None, 28, 28, 128)       74368     
                                                                 
 global_average_pooling2d (G  (None, 128)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                           

In [34]:
model_modified.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy'],
)

In [35]:
model_modified.fit(x_train, y_train, verbose = 2, batch_size = 64, epochs = 5)
model_modified.evaluate(x_test, y_test, batch_size = 64, verbose = 2) 

Epoch 1/5
938/938 - 17s - loss: 0.2500 - accuracy: 0.9407 - 17s/epoch - 18ms/step
Epoch 2/5
938/938 - 17s - loss: 0.1024 - accuracy: 0.9701 - 17s/epoch - 18ms/step
Epoch 3/5
938/938 - 18s - loss: 0.0873 - accuracy: 0.9743 - 18s/epoch - 19ms/step
Epoch 4/5
938/938 - 16s - loss: 0.0786 - accuracy: 0.9757 - 16s/epoch - 17ms/step
Epoch 5/5
938/938 - 16s - loss: 0.0704 - accuracy: 0.9787 - 16s/epoch - 17ms/step
157/157 - 2s - loss: 0.2569 - accuracy: 0.9144 - 2s/epoch - 10ms/step


[0.2569253742694855, 0.9143999814987183]

In [40]:
x = tf.random.normal(shape = (5, 299, 299, 3))
y = tf.constant([0, 1, 2, 3, 4])

In [45]:
#Using keras pretrained models:
inception_v3 = keras.applications.InceptionV3(include_top = False)
inception_v3.trainable = False 
base_input = inception_v3.layers[0].input
base_output = inception_v3.layers[-1].output
gap = layers.GlobalAveragePooling2D()(base_output)
dense1 = layers.Dense(512, activation = 'relu')(gap)
classification_v3 = layers.Dense(5)(dense1)
model_inception = Model(inputs = [base_input, ], outputs = [classification_v3, ])

# for layer in model_inception.layers[:-1]:
#     layer.trainable = False
    
print(model_inception.summary())

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_204 (Conv2D)            (None, None, None,   864         ['input_6[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_201 (Batch  (None, None, None,   96         ['conv2d_204[0][0]']             
 Normalization)                 32)                                                         

In [46]:
model_inception.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy'],
)

In [47]:
model_inception.fit(x, y, verbose = 2, batch_size = 64, epochs = 5)
# model_inception.evaluate(x_test, y_test, verbose = 2, batch_size = 64)

Epoch 1/5
1/1 - 4s - loss: 1.7155 - accuracy: 0.0000e+00 - 4s/epoch - 4s/step
Epoch 2/5
1/1 - 0s - loss: 1.9325 - accuracy: 0.2000 - 53ms/epoch - 53ms/step
Epoch 3/5
1/1 - 0s - loss: 1.1956 - accuracy: 0.4000 - 50ms/epoch - 50ms/step
Epoch 4/5
1/1 - 0s - loss: 1.1429 - accuracy: 0.8000 - 51ms/epoch - 51ms/step
Epoch 5/5
1/1 - 0s - loss: 1.1140 - accuracy: 0.6000 - 53ms/epoch - 53ms/step


In [51]:
# Using tensorflow hub models (Now integrated with Kaggle)
url = "https://www.kaggle.com/models/google/inception-v3/TensorFlow2/feature-vector/2"
feature_extractor = hub.KerasLayer(url, trainable=False, input_shape=(299, 299, 3))
feature_extractor.trainable = False
base_model_hub = Sequential([
    feature_extractor,
    layers.Dense(5),
])
print(base_model_hub.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 2048)              21802784  
                                                                 
 dense_12 (Dense)            (None, 5)                 10245     
                                                                 
Total params: 21,813,029
Trainable params: 10,245
Non-trainable params: 21,802,784
_________________________________________________________________
None


In [53]:
base_model_hub.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy'],
)

base_model_hub.fit(x, y, verbose=2, epochs = 5)

Epoch 1/5
1/1 - 6s - loss: 1.6733 - accuracy: 0.0000e+00 - 6s/epoch - 6s/step
Epoch 2/5
1/1 - 0s - loss: 1.5842 - accuracy: 0.0000e+00 - 54ms/epoch - 54ms/step
Epoch 3/5
1/1 - 0s - loss: 1.5279 - accuracy: 0.4000 - 51ms/epoch - 51ms/step
Epoch 4/5
1/1 - 0s - loss: 1.4693 - accuracy: 0.6000 - 53ms/epoch - 53ms/step
Epoch 5/5
1/1 - 0s - loss: 1.4036 - accuracy: 0.8000 - 55ms/epoch - 55ms/step
